In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install plotly 

In [3]:
# Import plotly library
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly
plotly.io.renderers.default = 'colab'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/My Drive/Data Analyst/HentaiVN/data

/content/drive/My Drive/Data Analyst/HentaiVN/data


In [6]:
# Read the data
data = pd.read_csv('cleaned_data.csv')

In [7]:
data.head()

,Name,Categories,Views,Date,Likes,Dislikes
0,Thời gian nhỏ đó bên cạnh (nguyên tác),"['Có che', 'HentaiVN', 'Nakadashi', 'Oneshot',...",6675,2020-08-12,31,0
1,Bộ Tứ Siêu Đẳng,"['Adult', 'Big Ass', 'Big Boobs', 'BlowJobs', ...",15599,2020-08-12,22,0
2,Sae-chan no Hatsutaiken 2 ~Katte ni Monzetsu S...,"['Anal', 'BBM', 'Full Color', 'Glasses', 'Loli...",24594,2020-08-12,42,7
3,[Doushoku (Onaka Emi)] #Uraaka Onee-san to Ich...,"['Big Boobs', 'BlowJobs', 'Breast Sucking', 'B...",43927,2020-08-11,117,3
4,Mutsumi-san no Hanshoku Katsudou Kiroku,"['Anal', 'Big Ass', 'Big Boobs', 'Bloomers', '...",23981,2020-08-11,37,6


In [8]:
# Preprocess the categories column
def remove_marks(s):
    res = ''
    for c in s:
        if c not in "'[]":
            res += c
    return res

def process_categories(s):
    res = s.split(', ')
    for i in range(len(res)):
        res[i] = remove_marks(res[i])
    return res

data['Categories'] = data['Categories'].apply(process_categories)

In [9]:
data = data[data['Views'] != 0]
data.reset_index(inplace=True)

# Insight about time

In [10]:
# We count the number of posts updated in every single day
date_plot = data['Date'].value_counts().sort_index()

In [11]:
date_plot.index = pd.to_datetime(date_plot.index)

In [12]:
# Sum the daily values into weekly values
weekly = date_plot.resample('W').sum()

In [13]:
fig = go.Figure(data=go.Scatter(x=weekly.index, y=weekly.values))
fig.show()

The weekly posts count seems to be random distributed

# Insight about most favorite hentais

Firstly, we need to define a metric to measure the favorite of a hentai manga. The formation will be (Likes - Dislikes) / Views * 100

In [14]:
# Add the favorite column
data['Favorite'] = data['Likes'] / (data['Likes'] + data['Dislikes'])

In [15]:
# Top 10 hentais that has the most favorite score and has more than 200k views (big enough)
top_favorite = data.sort_values(by='Favorite', ascending=False, ignore_index=True)
top_favorite = top_favorite[top_favorite['Views'] > 200000][:10]

In [16]:
top_favorite[['Name', 'Views', 'Date', 'Likes', 'Dislikes', 'Favorite']]

,Name,Views,Date,Likes,Dislikes,Favorite
47,Nàng hàng xóm [Full Color],673562,2019-08-19,654,2,0.996951
61,Lớp học của những thiếu niên hứng tình,381660,2020-01-31,664,3,0.995502
69,Buổi Ghé Thăm♥! (One punch Man),404081,2020-04-16,740,4,0.994624
82,Yêu là mù quáng,203541,2020-01-05,648,4,0.993865
86,Tình Yêu Chị Em 3 ( One Punch Man ),566535,2019-11-16,1600,10,0.993789
89,Sennen Reijou,250624,2020-08-04,626,4,0.993651
109,Địt chị gái phá tan cái nóng mùa hè!,208389,2020-02-02,715,5,0.993056
141,Cám dỗ của quỷ dữ,263136,2020-07-15,385,3,0.992268
143,Bệnh Yêu (NARUTO),273213,2020-07-12,256,2,0.992248
149,Cô dâu ngày thứ bảy,273228,2019-03-13,3003,24,0.992071


In [17]:
# The average views of one manga
avg_view = round(data['Views'].mean())
avg_view

200715

In [18]:
# The average favorite of one manga
avg_favorite = round(data['Favorite'].mean(), 2)
avg_favorite

0.87

In [19]:
# Plot the views of most favorite hentais
fig = go.Figure(go.Bar(
            x=top_favorite['Views'],
            y=top_favorite['Name'],
            orientation='h'))
fig.add_shape(dict(
            type="line",
            x0=avg_view,
            y0=-1,
            x1=avg_view,
            y1=10,
            line=dict(
                color="red",
                width=3,
                dash='dot'
            )
))
fig.add_annotation(
        x=200000,
        y=5,
        text="average view",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.show()

In [20]:
pd.Series(top_favorite['Categories'].sum()).value_counts()[:10]

Big Boobs    10
Big Penis     6
Romance       5
HentaiVN      5
Nakadashi     5
Stockings     5
Ahegao        5
BlowJobs      4
Virgin        4
Oneshot       4
dtype: int64

# Insight about most popularity hentais

In [41]:
# Top 10 hentais that has the most popularity
top_popularity = data.sort_values(by='Views', ascending=False, ignore_index=True)[:15]

In [42]:
top_popularity['Views'] = top_popularity['Views'].apply(lambda x : "{:,}".format(x))

In [43]:
top_popularity[['Name', 'Views', 'Likes', 'Dislikes', 'Favorite']]

,Name,Views,Likes,Dislikes,Favorite
0,Thánh Nôbita,"23,522,247",24388,3922,0.861462
1,Tuyển Tập Doraemon Doujinshi 18+,"20,047,194",22506,3980,0.849732
2,Mom’s Help,"18,347,545",4664,479,0.906864
3,Cột Thu Lôi,"14,168,698",10515,312,0.971183
4,Ceiling Girl,"14,055,496",51043,3360,0.938239
5,Tuyển tập Detective Conan Doujinshi 18+,"13,914,267",6336,2393,0.725856
6,Tôi Có Thể Thấy Số Lần Mà Họ Đã Lên Đỉnh!,"13,035,213",43137,1164,0.973725
7,Doraemon và bí mật động trời,"12,991,189",19375,27290,0.415193
8,Chàng Thám Tử Đào Hoa,"11,278,016",38511,930,0.976420
9,Traditional Job of Washing Girls' Body,"11,129,841",27444,741,0.973709


In [46]:
# Plot the views of most favorite hentais
fig = go.Figure(go.Bar(
            x=top_popularity['Views'],
            y=top_popularity['Name'],
            orientation='h'))
fig.show()

# The correlation of views and favorite score

In [25]:
# Calculate the correlation
corr = data[['Views', 'Favorite']].corr()['Views'][1]
corr

-0.03629931730260608

In [26]:
fig = go.Figure(data=go.Scatter(x=data['Views'], y=data['Favorite'], mode='markers'))
fig.show()

## Insight about genres
We're gonna use weighted average method to calculate the average favorite score for each genre

In [27]:
genre_list = np.unique(data['Categories'].sum())


In [28]:
len(genre_list)

162

In [29]:
# Create genre dataframe for further analysis
columns = {column:[0 for i in range(len(genre_list))] for column in ['Total_views', 'Count', 'Average_views', 'Views*Favorite', 'Average_favorite']}
genre_df = pd.DataFrame(columns, index=genre_list) 
genre_df.head()

,Total_views,Count,Average_views,Views*Favorite,Average_favorite
3D Hentai,0,0,0,0,0
Action,0,0,0,0,0
Adult,0,0,0,0,0
Adventure,0,0,0,0,0
Ahegao,0,0,0,0,0


In [30]:
# fill data
for i in range(len(data)):
    for genre in data['Categories'][i]:
        genre_df['Total_views'][genre] += data['Views'][i]
        genre_df['Count'][genre] += 1
        genre_df['Views*Favorite'][genre] += data['Views'][i] * data['Favorite'][i]

In [37]:
# Continue to fill data for full table
genre_df['Average_views'] = genre_df['Total_views'] / genre_df['Count']
genre_df['Average_favorite'] = genre_df['Views*Favorite'] / genre_df['Total_views']

genre_df['Average_views'] = genre_df['Average_views'].apply(lambda x : "{:,.0f}".format(x))
genre_df['Average_favorite'] = genre_df['Average_favorite'].apply(lambda x : "{:,.2f}".format(x))
genre_df['Total_views'] = genre_df['Total_views'].apply(lambda x : "{:,.0f}".format(x))

In [38]:
genre_df.head()

,Total_views,Count,Average_views,Views*Favorite,Average_favorite
3D Hentai,"128,263,321",83,"1,545,341",109274751,0.85
Action,"186,520,345",286,"652,169",146837512,0.79
Adult,"272,916,540",336,"812,252",235805683,0.86
Adventure,"92,240,003",117,"788,376",82955012,0.90
Ahegao,"1,379,544,705",5154,"267,665",1160915327,0.84


In [33]:
genre_topfavor = genre_df.sort_values(by='Average_favorite', ascending=False)
genre_topfavor[['Count', 'Average_views', 'Average_favorite']][:15]

,Count,Average_views,Average_favorite
Webtoon,56,"2,311,801",0.96
Vanilla,2612,"144,819",0.95
Kuudere,60,"453,465",0.94
Brocon,420,"193,387",0.93
Imouto,270,"235,956",0.93
Cousin,224,"403,430",0.93
Twintails,372,"247,098",0.92
Cunnilingus,654,"331,198",0.92
Nun,31,"435,207",0.91
Romance,3386,"215,864",0.91


In [40]:
genre_topgross = genre_df.sort_values(by='Average_favorite')[:10]
genre_topgross[['Total_views', 'Count', 'Average_views', 'Average_favorite']]

,Total_views,Count,Average_views,Average_favorite
Guro,"69,513,789",297,"234,053",0.53
Scat,"30,722,732",125,"245,782",0.56
Horror,"51,525,176",139,"370,685",0.59
Ryona,"60,150,749",170,"353,828",0.60
Dirty,"48,355,309",128,"377,776",0.62
Insect (Côn Trùng),"23,948,920",82,"292,060",0.64
Furry,"52,734,567",357,"147,716",0.71
Time Stop,"33,273,130",48,"693,190",0.71
Bestiality,"111,459,851",309,"360,711",0.72
Father,"57,494,510",88,"653,347",0.73


In [39]:
genre_df.sort_values(by='Total_views', ascending=False)[:20]
genre_df.loc[['Big Boobs', 'Anal', 'Big Ass', 'Rape', 'Big Penis', 'SchoolGirl', 'Incest']][['Total_views', 'Count', 'Average_views']]

,Total_views,Count,Average_views
Big Boobs,"2,900,492,498",11927,"243,187"
Anal,"1,186,683,554",4105,"289,082"
Big Ass,"1,162,405,834",3101,"374,849"
Rape,"1,050,736,403",3266,"321,720"
Big Penis,"952,744,656",2910,"327,404"
SchoolGirl,"940,451,290",4187,"224,612"
Incest,"920,238,075",2952,"311,734"


In [36]:
genre_df.loc['Incest']

Total_views         920238075
Count                    2952
Average_views         311,734
Views*Favorite      791110898
Average_favorite         0.86
Name: Incest, dtype: object